## Duke Qualtrics Survey Data File Processing Guide



#### Load Packages

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

#### Load Data File (Download from Duke Qualtrics Platform)

In [2]:
df = pd.read_csv("Mapping_Survey11_19.csv")

### Phase 1:

Separate the Mapping Data and Analysis Data

In [3]:
def phase1clean(df):
    mapping_data = df.loc[:,'Q1':'Q6'].copy()
    analyze_data = df.loc[:,'Q21':].copy()
    mapping_data.dropna(axis = 0, how='all',inplace=True)
       
    return mapping_data, analyze_data

In [4]:
mapping, analysis = phase1clean(df)

### Phase 2:

Merge survey answers in "other" sections of a question with other selected answers of that question
Remove Mapping Data before a specific date (06/19/2018)

In [5]:
def phase2clean(df):
    
    def cleanOtro(row):
        for index,item in row.items():
            if isinstance(item, str):
                if "Otro" in item:
                    if not pd.isna(row[index]):
                        list_ = row[index].split(',')
                    list_.pop()
                    if isinstance(row[row.index.get_loc(index)+1], str):
                        list_.append(str.translate(row[row.index.get_loc(index)+1],str.maketrans(',','/')))
                    row[index] = ",".join(list_)
                    row[row.index.get_loc(index)+1] = np.nan
        return row
    
    df = df.apply(cleanOtro,axis=1)
    df.drop('Q19_9_TEXT',inplace=True,axis=1)
    df.drop('Q2_TEXT',inplace=True,axis=1)
    df.drop(1, axis=0,inplace=True)
    
    # Remove data before the 61st row
    df = df.loc[61:,:]
    return df

In [6]:
mapping = phase2clean(mapping)

### Fix the names of categories

The Activity Category section of the survey is modified to be Javascript code for better display. Change the corresponding columns back to the name of categories.

In [7]:
def fix_categories(row):
    
    Categories = ['Primer Nivel de Atención','Educación para el Cambio de Comportamiento','Agua potable y saneamiento ambiental','Disponibilidad y Economía Familiar']
    new_categories = []
    if not pd.isna(row['Q27_1']):
        new_categories.append(Categories[0])
    if not pd.isna(row['Q27_30']):
        new_categories.append(Categories[1])
    if not pd.isna(row['Q27_31']):
        new_categories.append(Categories[2])
    if not pd.isna(row['Q27_32']):
        new_categories.append(Categories[3])

    return new_categories

### Group answer columns

The data file provided by Duke Qualtrics records every answer in a separate column. This function is used to put answers of the same question into one group.

In [8]:
def sortByGroup(lst):
    groups = []
    for item in lst:
        match = False
        for g in range(len(groups)):
            group = groups[g]
            parent = group[0]
            try:
                if parent[:4] == item[:4]:
                    
                    if parent[-1] == ')':
                        if 'TEXT' in item and 'TEXT' not in parent:
                            pass
                        else:
                            if parent[-3:] == item[-3:]:
                                group.append(item)
                                match = True
                    else:
                        group.append(item)
                        match = True
            except:
                pass
        if not match:
            groups.append([item])
    return groups

In [9]:
# String processing tools

def make_string_list(series):
    lst = []
    if len(series) == 0:
        return lst
    
    for i in range(len(series)):
        lst.append(series[i])
    return lst

def make_string(series):
    series = series.dropna()
    lst = []
    if len(series) == 0:
        return 'unknown'
    
    for i in range(len(series)):

        lst.append(str(series[i]))
    
    if len(lst) > 1:
        return ','.join(lst)
    else:
        return lst[0]

### Phase 3:

Apply various steps to each response in the mapping data file. Transform each response into multiple rows of different Municipios.

In [10]:
def clean_row_new(row):
    
    
    categories = fix_categories(row)
    
    firstpart = row[:5]
    lastpart = row[-25:]
    middlepart = row[5:-25]
    middlepart = middlepart.drop(['Q27_1','Q27_30','Q27_31','Q27_32'])
    middlepart = middlepart.dropna()
   
    row = firstpart.append(middlepart).append(lastpart)

    Groups = sortByGroup(row.index.values)
    
    df = pd.DataFrame(columns = ['NameOrg','TypeOrg','NameProg','StartDate','EndDate',
                        'Category','Activity','Area','Municipio',
                        'Beneficiaries','NumBeneficiaries','EduIntervention(Y/N)','EduIntervention',
                       'Place','Nutritionist','Goal'])
    temp = pd.Series(index=['NameOrg','TypeOrg','NameProg','StartDate','EndDate',
                        'Category','Activity','Area','Municipio',
                        'Beneficiaries','NumBeneficiaries','EduIntervention(Y/N)','EduIntervention',
                       'Place','Nutritionist','Goal'])

    index = 4

    num_categories = len(categories)
    current_index = index+num_categories+1
    for i in range(num_categories):
        temp['Goal'] = make_string(row[Groups[-1]])
        temp['Nutritionist'] =  make_string(row[Groups[-2]])
        temp['Place'] =  make_string(row[Groups[-3]])
        temp['EduIntervention'] =  make_string(row[Groups[-4]])
        temp['EduIntervention(Y/N)'] =  make_string(row[Groups[-5]])
        temp[:5] = row[:5]
        temp['Category'] = categories[i]
        
        activities = make_string_list(row[Groups[index+i+1]])
        num_activities = len(activities)
    
        for j in range(num_activities):
            
            temp['Activity'] = activities[j]
            areas = make_string_list(row[Groups[current_index]])
            num_areas = len(areas)
            current_index += 1
            
            if current_index >= len(Groups):
                return df
            
            beneficiaries = make_string_list(row[Groups[current_index]])

            temp['Beneficiaries'] = make_string(row[Groups[current_index]])
            num_beneficiaries = len(beneficiaries)
            current_index += 1
            
            if current_index >= len(Groups):
                return df           
            
            for k in range(num_areas):
                temp[7] = areas[k]
                if pd.isna(row[current_index]):
                    return
       
                municipios = make_string_list(row[Groups[current_index]])
                num_municipios = len(municipios)
                current_index += 1
                
                if current_index >= len(Groups):
                    return df
                
                for l in range(num_municipios):
                   
                    temp['Municipio'] = municipios[l]
                    if 'TEXT' in Groups[current_index][0]:
                        temp['NumBeneficiaries'] = make_string(row[Groups[current_index]])
                        current_index += 1
                    else:
                        temp['NumBeneficiaries'] = 'unknown'
                    
                    df = df.append(temp,ignore_index=True)         
    return df


In [11]:
result = pd.DataFrame(columns = ['NameOrg','TypeOrg','NameProg','StartDate','EndDate',
                        'Category','Activity','Area','Municipio',
                        'Beneficiaries','NumBeneficiaries','EduIntervention(Y/N)','EduIntervention',
                       'Place','Nutritionist','Goal'])

In [12]:
for i in range(len(mapping)):
    print("Processing response {}..".format(i))
    new_df = clean_row_new(mapping.iloc[i,:])
    result = result.append(new_df)

Processing response 0..
Processing response 1..
Processing response 2..
Processing response 3..
Processing response 4..
Processing response 5..
Processing response 6..
Processing response 7..
Processing response 8..
Processing response 9..
Processing response 10..
Processing response 11..
Processing response 12..
Processing response 13..
Processing response 14..
Processing response 15..
Processing response 16..
Processing response 17..
Processing response 18..
Processing response 19..
Processing response 20..
Processing response 21..
Processing response 22..
Processing response 23..
Processing response 24..
Processing response 25..
Processing response 26..
Processing response 27..
Processing response 28..
Processing response 29..
Processing response 30..
Processing response 31..
Processing response 32..
Processing response 33..
Processing response 34..
Processing response 35..
Processing response 36..
Processing response 37..
Processing response 38..
Processing response 39..
Processing

### Add Geocodes to the output file

In [13]:
geocodes = pd.read_csv("geocodes.csv")
result = result.merge(geocodes.drop('Country',axis=1),how = 'left',on=['Area','Municipio'],left_index=True)
cols = result.columns.tolist()
cols.insert(9,cols[-1])
cols.pop()
cols.insert(9,cols[-1])
cols.pop()
result = result[cols]

### Add respond date to the output file

In [14]:
Data = pd.DataFrame(data=[df['StartDate'],df['EndDate'],df['Q1']]).T

In [15]:
Data = Data.rename({'Q1':'NameOrg','StartDate':'RespondStart','EndDate':'RespondEnd'},axis=1)
Data = Data.dropna()
Data = Data.reset_index(drop=True)
pass

In [16]:
result = Data.merge(result,on='NameOrg')

#### Preview of the output file

In [18]:
#result

### Save output file into a .csv file

In [30]:
result.to_csv("mapping_file_11_19.csv",encoding = 'utf-8')

-----

## Analysis file

In [19]:
def join_text(row):
    new_series = pd.Series()
    groups = sortByGroup(row.index.values)
    for group in groups:
        lst = []
        for item in row[group]:
            if not pd.isna(item):
                lst.append(str(item))
        new_series[group[0][:4]] = ','.join(lst)
    return new_series

In [20]:
analysis = pd.concat([pd.DataFrame(mapping.iloc[:,:5]),pd.DataFrame(analysis)],axis=1)

In [21]:
def clean_analysis(analysis):
    new_df = pd.DataFrame()
    df = pd.concat([pd.DataFrame(mapping.iloc[:,:5]),pd.DataFrame(analysis.iloc[:,:-3])],axis=1)
    for i in range(len(df)):
        new_series = join_text(df.iloc[i,:])
        new_df= new_df.append(new_series,ignore_index=True)
    new_df =new_df[new_series.index.values]
    return new_df

analysis_file = clean_analysis(analysis)

#### Preview of the output file

In [23]:
#analysis_file

### Save output file into a .csv file

In [ ]:
analysis_file.dropna(how='all',axis=0)
analysis_file.to_csv("Analysis_11_19.csv")